In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

**flatmap** vs. **map** only affects how the data is collected.

## Fold, Aggregate, countByValue, takeSample, and foreach(n)


### Fold - similiar to a reduce function
**fold** is similar to reduce with a default value. 
- takes function with same design as needed for reduce()
- Takes a default value to be used for the intial call on each partition
- Returns a new value of hte same type

---

If there's 3 partitions

`.fold(0,lambda x,y : x+y)`

Step 1 : **0** 1 3 5 | **0** 7 9 | **0**

Step 2 : **0** 9 | **0** 16

Step 3 :25

---

`.fold(1,lambda x,y : x+y)`

If there's 3 partitions

Step 1 : **1** 1 3 5 | **1** 7 9 | **1**

Step 2 : **0** 9 | **0** 16

Step 3 : 25

Fold is often used for error handling. Handle error on your own. The 0 value will automatically handle the error. 



#### Lets look at odd numbers, want to calculate (sum / count)

In [15]:
odd_nums = sc.parallelize([1,3,5,7,9],2)

In [23]:
odd_nums.map(lambda x : x+1 if x == 3 else x).collect()

[1, 4, 5, 7, 9]

In [24]:
odd_nums.map(lambda x : x+1 if x == 3 else x+2 if x==5 else 0).collect()

[0, 4, 7, 0, 0]

In [16]:
odd_nums.collect()

[1, 3, 5, 7, 9]

We will make a 2-value tuple `(0,0)` which will be considered as `x`.

- `x[0]` will be our sum
- `x[1]` will be our count

```python
.aggregate((0,0),  # default value
    lambda x,y : (x[0],x[1]+y),  # within a partition
    lambda x,y: (x[0]+y[0],x[1]+y[1])) # condensing partitions
```

#### Lets look at the parts

##### Default Value
Will default to (0,0) as a starting point. Then will continually add. 

##### Within partition 

`lambda x,y : (x[0]+1,x[1]+y)`

Let's work through this. 

1. We start with `(0,0) = x`
2. We encounter our first value 1 so `y=1`
3. We want (0,0) to be => (1,1)


##### Gathering Partitions results

`lambda x,y: (x[0]+y[0],x[1]+y[1]))`

Let's work through this step by step. Lets say the partition break is `[1,3] [5,7,9]`

Then after applying the function within the partition we should have `(4,2) (21,3)`

So to reiterate:

1. `(4,2) = x`
2. `(21,3) = y`
3. so the lambda function should add these pair-wise together
4. `x[0]+y[0] = 25`
5. `x[1]+y[1] = 5`
6. Returns `(25,5)`







#### Example 4-1 for the numbers between 1 and 9 calculate sum of odd numbers

In [20]:
odd_nums.reduce(lambda x,y : x + y)

25

#### Example 4-2 for the numbers between 1 and 9 calculate the sum of the odd numbers using fold

In [21]:
odd_nums.fold(0,lambda x,y : x + y)

25

#### Example 4-3 using aggregate () return (sum # of elements ) of odd numbers

In [22]:
odd_nums.aggregate((0,0),lambda x,y:(x[0]+1,+x[1]+y),lambda x,y:(x[0]+y[0],x[1]+y[1]))

(5, 25)

In [17]:
odd_nums.glom().collect()

[[1, 3], [5, 7, 9]]

### Whats the difference between take() and first()?

One returns values, the other returns a string

In [25]:
odd_nums = sc.parallelize([1,3,5,7,9],2)

In [26]:
odd_nums.take(1)

[1]

In [30]:
odd_nums.first()

1

### Whats the difference between sample() and takeSample()?


### Sample
Sample(withReplacement=True, fraction=0.5, seed=1) **transformation**
- **CREATES A NEW RDD** with random elements form the calling RDD
- with replacement (with repeats)
- fraction - expected positive
- expected probability element is used
- seed (for standardizing randomiztion) - otherwise default based on millisecond time

### takeSample
takesSample (with replacement , num, seed) : ** action **
- returns fixed size sample subset of an RDD as an **ARRAY**
- with replacement allow sample multiple times
- num - example number of sampled element
- seed


### Example 5-1

Try collect(), count(), countByValue(), top(n), take(n), first(), takeSample(), operations on Z

In [34]:
x = sc.parallelize([3,4,1,2])
y = sc.parallelize(range(2,6))
z = x.union(y)

In [35]:
z.collect()

[3, 4, 1, 2, 2, 3, 4, 5]

In [36]:
z.count()

8

In [37]:
z.countByValue()

defaultdict(int, {1: 1, 2: 2, 3: 2, 4: 2, 5: 1})

In [38]:
z.top(3)

[5, 4, 4]

In [40]:
z.take(3)

[3, 4, 1]

In [41]:
z.first()

3

In [47]:
z.takeSample(withReplacement=False,num=3,seed=None)

[4, 4, 5]

In [51]:
z.takeSample(withReplacement=True,num=20,seed=None)

[1, 5, 4, 5, 4, 2, 2, 5, 4, 2, 2, 2, 3, 4, 2, 5, 4, 3, 2, 4]

In [60]:
z.glom().collect()

[[], [3], [], [4], [], [1], [], [2], [], [2], [], [3], [], [4], [], [5]]

In [59]:
res = z.sample(True,0.5)
res.glom().collect()

[[], [3, 3], [], [4], [], [], [], [], [], [], [], [3], [], [], [], []]

In [65]:
b = z.foreach(lambda x : x+1)

## Week 3 - Pair RDDs

Pair RDDs are a **key value pair**. 

|Key|value|
|---|------------------------------|
|001| 'some data that i have added'|

Very common structure for schemaless data or NoSQL. In NoSQL - key is unique. Spark, key could be duplicated. Key could show up in multiple places

#### Keys 
Could be simple, or complex objects (tuples)
Could be simple, or a complex json

#### Exercise 1 - get words from readme file

In [67]:
path = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/README.md'
rdm = sc.textFile(path)

In [82]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,1))
words.collect()[:20]

[(u'#', 1),
 (u'Apache', 1),
 (u'Spark', 1),
 (u'Spark', 1),
 (u'is', 1),
 (u'a', 1),
 (u'fast', 1),
 (u'and', 1),
 (u'general', 1),
 (u'cluster', 1),
 (u'computing', 1),
 (u'system', 1),
 (u'for', 1),
 (u'Big', 1),
 (u'Data.', 1),
 (u'It', 1),
 (u'provides', 1),
 (u'high-level', 1),
 (u'APIs', 1),
 (u'in', 1)]

## Transformations designed for key value pairs

|function | description |
|-----|------------------|
|keys()| gets the keys|
|values()| value|

#### Do the same but do value as length of word and sort by value

In [89]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,len(x)))
sorted_list = words.sortBy(lambda x: x[1],ascending=False)
sorted_list.collect()[:20]

[(u'[IntelliJ](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools#UsefulDeveloperTools-IntelliJ).',
  115),
 (u'[Eclipse](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools#UsefulDeveloperTools-Eclipse)',
  112),
 (u'Version"](http://spark.apache.org/docs/latest/building-spark.html#specifying-the-hadoop-version)',
  96),
 (u'3"](https://cwiki.apache.org/confluence/display/MAVEN/Parallel+builds+in+Maven+3).',
  82),
 (u'tests](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools).',
  81),
 (u'Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 65),
 (u'Guide](http://spark.apache.org/docs/latest/configuration.html)', 62),
 (u'wiki](https://cwiki.apache.org/confluence/display/SPARK).', 57),
 (u'page](http://spark.apache.org/documentation.html)', 49),
 (u'sc.parallelize(range(1000)).count()', 35),
 (u'Maven](http://maven.apache.org/).', 33),
 (u'<http://spark.apache.org/>', 26),
 (u'MASTER=spark://host:7077', 

#### Example 3 

Create a pair RDD with (length of a word, list of words) from README.md

In [102]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (len(x),x))
res = words.groupByKey().map(lambda x: (x[0],list(x[1])))
res.sortByKey()
res.collect()


[(96,
  [u'Version"](http://spark.apache.org/docs/latest/building-spark.html#specifying-the-hadoop-version)']),
 (112,
  [u'[Eclipse](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools#UsefulDeveloperTools-Eclipse)']),
 (2,
  [u'is',
   u'It',
   u'in',
   u'R,',
   u'an',
   u'It',
   u'of',
   u'##',
   u'on',
   u'##',
   u'is',
   u'To',
   u'do',
   u'to',
   u'do',
   u'if',
   u'by',
   u'-T',
   u'in',
   u'is',
   u'at',
   u'an',
   u'##',
   u'to',
   u'is',
   u'to',
   u'##',
   u'if',
   u'##',
   u'in',
   u'To',
   u'of',
   u'Pi',
   u'to',
   u'to',
   u'be',
   u'or',
   u'to',
   u'on',
   u'to',
   u'or',
   u'to',
   u'an',
   u'if',
   u'is',
   u'in',
   u'of',
   u'if',
   u'no',
   u'##',
   u'is',
   u'be',
   u'on',
   u'to',
   u'or',
   u'##',
   u'to',
   u'to',
   u'in',
   u'of',
   u'to',
   u'at',
   u'on',
   u'of',
   u'##',
   u'to',
   u'in',
   u'an',
   u'on',
   u'to']),
 (4,
  [u'fast',
   u'APIs',
   u'that',
   u'data',

### mapValues

Pass each value in the key value pair RDD through a map function wihtout changing the keys

#### Example 4

In [116]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,1))
res = words.groupByKey()
res.collect()[:3]

[(u'when', <pyspark.resultiterable.ResultIterable at 0x107e3cc10>),
 (u'R,', <pyspark.resultiterable.ResultIterable at 0x107e2b0d0>),
 (u'including', <pyspark.resultiterable.ResultIterable at 0x107e2b150>)]

In [118]:
res2 = res.mapValues(sum)
res2.sortBy(lambda x: x[1], ascending = False).collect()[:10]

[(u'the', 22),
 (u'Spark', 15),
 (u'to', 14),
 (u'for', 11),
 (u'and', 11),
 (u'##', 8),
 (u'a', 8),
 (u'run', 7),
 (u'can', 7),
 (u'is', 6)]

### flatMapValues

#### Example 5 - similiar to before, but using flatMapValues


In [1]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (len(x),x))
words.flatMapValues()

NameError: name 'rdm' is not defined

### reduceByKey

similar to reduce ()
reuns in parallel reduce operations

#### Example 6 - do the word counts - generate word / occurance again

In [132]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,1))
res = words.reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False)
res.collect()[:10]

[(u'the', 22),
 (u'Spark', 15),
 (u'to', 14),
 (u'for', 11),
 (u'and', 11),
 (u'##', 8),
 (u'a', 8),
 (u'run', 7),
 (u'can', 7),
 (u'is', 6)]

### Which requires less shuffles?

    .groupByKey().mapValues(lambda x : sum(x))
    - Sends all the values over first, then condenses 2nd with the mapValues

    .reduceByKey(lambda x,y: x+y)
    - Does partition accumulation first, then send pair to be combined
    - FEWER SHUFFLES